In [ ]:
!pip install JACK-Client
!pip install soundfile

In [5]:
#Import libraries
import numpy as np
import pandas as pd
import scipy as sc
import matplotlib.pyplot as plt
import jack
import wave
import sys
import os
import threading
import soundfile as sf
import time

In [58]:
#Version final de CCVPHAT
def localization(filex, filey, window, distm, hann):
    """
    This function implements ccvphat procedure for sound source localitation
    utilizing only 2 microphones in real time
    filex = Audio file for first microphone 
    filey = Audio file for second microphone
    window = size of sample window we are going to use
    distm = distance between microphones
    hann = 0,1. If 0 dont use hann, if 1, use hann
    
    """
    xa = sf.read(filex)  
    ya = sf.read(filey)
    #sf.read returns file, frames, the values are on the first entry
    xav = xa[0]  
    yav = ya[0]
    venti = 0
    vento = window
    aal = len(xav)
    angulos = []
    #samp is the maximum sample delay given physical limits
    #We calculate the maximum delay in samples taking wind_velocity = 343, supossing the medium is air
    #And we also use de distance between microphones and frames per second
    samp = int(np.floor((distm *xa[1])/ (343)))
    while venti < aal: 
        #Reading the signal by frame windows changing each iteration
        xav1 = xav[venti:vento]
        yav1 = yav[venti:vento]
        #We center the signals
        mx = np.mean(xav1)
        my = np.mean(yav1)
        for i in range(window):
            xav1[i] = xav1[i] - mx
            yav1[i] = yav1[i] - my
        if hann == 1:
            xav1[i] = 0.5*(1-np.cos(2*np.pi*xav1[i] /(1024)))  #this is periodic hann
            yav1[i] = 0.5*(1-np.cos(2*np.pi*yav1[i] /(1024)))
        else:
            pass
        #We apply fourier transformation to both signals
        xfaira1 = np.fft.fft(xav1)
        yfaira1 = np.fft.fft(yav1)
        #We calculate generalized cross correlation with fase transformation
        ccvfpaira1 = (xfaira1*np.conj(yfaira1))/(np.abs(xfaira1*np.conj(yfaira1)))
        ccvfpaira1[np.isnan(ccvfpaira1)] = 0  #We alter the first value because is 0 and causes mistakes
        #Return to time domain
        ccvpaira1 = np.real(np.fft.ifft(ccvfpaira1))  
        #Maximum value
        #Depending on the window we limit the possible values of our array
        valoresp = np.concatenate((ccvpaira1[0:samp],ccvpaira1[window-samp:window-1])) #Possible values
        maxpaira = np.max(valoresp) #Searching for maximum value possible
        indicepaira = np.where(valoresp == maxpaira) #we search de index of that value in our array
        #We calculate the delay frame considering it can be at the beggining or end
        if indicepaira[0][0]<len(valoresp)/2:
            delay = indicepaira[0][0] - 1
        else:
            delay = len(valoresp)  - indicepaira[0][0] -1  
        t = (delay/48000) #our delay in seconds
        #To calculate the angle we use the relation: sin(theta)=V_sound * t_2-1 /d
        angulo = np.rad2deg(np.arcsin((343*t)/distm)) #We also change the angle to degrees in this step.
        angulos = angulos + [angulo]
        #We have to change values of loop in order to advance through the whole file
        venti = venti + window
        vento = vento + window
    print("Quantity of angles obtained:", len(angulos))
    angulof = np.mean(angulos) #We calculate the mean of angles saved from all windows
    print("Phi is equal to:", angulof)
    print("Theta is equal to:", 90-angulof)

In [75]:
localization('wav_mic1.wav', 'wav_mic2.wav', 1024, 0.18, 0) #no noice, no hann, window=1024, angle=60

<ipython-input-58-83601084fe4c>:45: RuntimeWarning: invalid value encountered in true_divide
  ccvfpaira1 = (xfaira1*np.conj(yfaira1))/(np.abs(xfaira1*np.conj(yfaira1)))


Quantity of angles obtained: 1502
Theta is equal to: 25.092325013445862
Phi is equal to: 64.90767498655414


In [60]:
localization('wav_mic1.wav', 'wav_mic2.wav', 2048, 0.18, 0) #no noice, no hann, window=2048 angle=60

<ipython-input-58-83601084fe4c>:45: RuntimeWarning: invalid value encountered in true_divide
  ccvfpaira1 = (xfaira1*np.conj(yfaira1))/(np.abs(xfaira1*np.conj(yfaira1)))


Quantity of angles obtained: 751
Theta is equal to: 29.0366734010125
Phi is equal to: 60.9633265989875


In [61]:
localization('wav_mic1.wav', 'wav_mic2.wav', 1024, 0.18, 1) #no noice, with hann, window=1024 angle=60

Quantity of angles obtained: 1502
Theta is equal to: 22.376430908146613
Phi is equal to: 67.62356909185338


In [63]:
localization('wav_mic1n.wav', 'wav_mic2n.wav', 1024, 0.18, 0) #with noice, mo hann, window=1024 angle=60

<ipython-input-58-83601084fe4c>:45: RuntimeWarning: invalid value encountered in true_divide
  ccvfpaira1 = (xfaira1*np.conj(yfaira1))/(np.abs(xfaira1*np.conj(yfaira1)))


Quantity of angles obtained: 1502
Theta is equal to: -0.38333790804503093
Phi is equal to: 90.38333790804504


In [65]:
localization('wav_mic1n.wav', 'wav_mic2n.wav', 1024, 0.18, 1) #with noice, with hann, window=1024 angle=60

Quantity of angles obtained: 1502
Theta is equal to: -0.9064358593743849
Phi is equal to: 90.90643585937438


In [66]:
#Trying with other files
localization('wav_mic190.wav', 'wav_mic290.wav', 1024, 0.18, 0) #no noice, no hann, window=1024 angle=90

<ipython-input-58-83601084fe4c>:45: RuntimeWarning: invalid value encountered in true_divide
  ccvfpaira1 = (xfaira1*np.conj(yfaira1))/(np.abs(xfaira1*np.conj(yfaira1)))


Quantity of angles obtained: 1502
Theta is equal to: 0.17178751650943988
Phi is equal to: 89.82821248349056


In [67]:
#Trying with other files
localization('wav_mic190.wav', 'wav_mic290.wav', 2048, 0.18, 0) #no noice, no hann, window=2048 angle=90

<ipython-input-58-83601084fe4c>:45: RuntimeWarning: invalid value encountered in true_divide
  ccvfpaira1 = (xfaira1*np.conj(yfaira1))/(np.abs(xfaira1*np.conj(yfaira1)))


Quantity of angles obtained: 751
Theta is equal to: -0.011104392502885125
Phi is equal to: 90.01110439250289


In [68]:
#Trying with other files
localization('wav_mic190n.wav', 'wav_mic290n.wav', 1024, 0.18, 0) #with noice, no hann, window=2048 angle=90

<ipython-input-58-83601084fe4c>:45: RuntimeWarning: invalid value encountered in true_divide
  ccvfpaira1 = (xfaira1*np.conj(yfaira1))/(np.abs(xfaira1*np.conj(yfaira1)))


Quantity of angles obtained: 1504
Theta is equal to: -0.721644076855527
Phi is equal to: 90.72164407685553


In [69]:
#Trying with other files
localization('wav_mic190n.wav', 'wav_mic290n.wav', 1024, 0.18, 1) #with noice, with hann, window=1024 angle=90

Quantity of angles obtained: 1504
Theta is equal to: -0.4094482585564652
Phi is equal to: 90.40944825855647


In [70]:
#Trying with other files
localization('wav_mic190.wav', 'wav_mic290.wav', 1024, 0.18, 1) #no noice, with hann, window=1024 angle=90

Quantity of angles obtained: 1502
Theta is equal to: 0.340237782425113
Phi is equal to: 89.6597622175749


In [71]:
#Trying with other files
localization('wav_mic190.wav', 'wav_mic290.wav', 2048, 0.18, 1) #no noice, with hann, window=2048 angle=90

Quantity of angles obtained: 751
Theta is equal to: 0.048024023829560135
Phi is equal to: 89.95197597617044


In [72]:
#Trying with other files that have more than one signal
localization('wav_mic1v.wav', 'wav_mic2v.wav', 1024, 0.18, 0) #no noice, no hann, window=1024 angle=-30, 90

<ipython-input-58-83601084fe4c>:45: RuntimeWarning: invalid value encountered in true_divide
  ccvfpaira1 = (xfaira1*np.conj(yfaira1))/(np.abs(xfaira1*np.conj(yfaira1)))


Quantity of angles obtained: 1503
Theta is equal to: 2.303606467142043
Phi is equal to: 87.69639353285795


In [74]:
#Trying with other files that have more than one signal
localization('wav_mic1v.wav', 'wav_mic2v.wav', 1024, 0.18, 1) #no noice, with hann, window=2048 angle=-30, 90

Quantity of angles obtained: 1503
Theta is equal to: 2.5400493357268363
Phi is equal to: 87.45995066427317
